In [1]:
import numpy as np
import pandas as pd
import gzip
import csv
import os

np.set_printoptions(precision=2)
np.random.seed(123)

In [2]:
# generate a dictionary of data grouped by sequenceID
def gen_data_dict(file_path):
    with gzip.open(file_path, 'rt') as file:
        df = pd.read_csv(file)

    _dict = tuple(df.groupby('sequenceID'))
    return _dict

# test
seqs   = gen_data_dict('sequence_label_data/signals.gz')
labels = gen_data_dict('sequence_label_data/labels.gz')

In [3]:
def get_data(i, seqs, labels):
    # sequence
    sequence = seqs[i][1]['logratio'].to_numpy()
    sequence = np.append([0], sequence)

    # labels
    lab_df = labels[i][1]

    # get label sets
    lab_df_1, lab_df_2 = lab_df[lab_df['fold'] == 1], lab_df[lab_df['fold'] == 2]

    pos_lab_df_1, pos_lab_df_2 = lab_df_1[lab_df_1['changes'] == 1], lab_df_2[lab_df_2['changes'] == 1]
    neg_lab_df_1, neg_lab_df_2 = lab_df_1[lab_df_1['changes'] == 0], lab_df_2[lab_df_2['changes'] == 0]

    neg_start_1, neg_end_1 = neg_lab_df_1['start'].to_numpy(), neg_lab_df_1['end'].to_numpy()
    pos_start_1, pos_end_1 = pos_lab_df_1['start'].to_numpy(), pos_lab_df_1['end'].to_numpy()
    neg_start_2, neg_end_2 = neg_lab_df_2['start'].to_numpy(), neg_lab_df_2['end'].to_numpy()
    pos_start_2, pos_end_2 = pos_lab_df_2['start'].to_numpy(), pos_lab_df_2['end'].to_numpy()

    return sequence, neg_start_1, neg_end_1, pos_start_1, pos_end_1, neg_start_2, neg_end_2, pos_start_2, pos_end_2

# test
seq_id = 186
sequence, neg_start_1, neg_end_1, pos_start_1, pos_end_1, neg_start_2, neg_end_2, pos_start_2, pos_end_2 = get_data(seq_id, seqs, labels)
print("sequence:    ", sequence[0:5])
print("neg start 1: ", neg_start_1)
print("neg end 1    ", neg_end_1)
print("pos start 1: ", pos_start_1)
print("pos end 1:   ", pos_end_1)
print("neg start 2: ", neg_start_2)
print("neg end 2    ", neg_end_2)
print("pos start 2: ", pos_start_2)
print("pos end 2:   ", pos_end_2)

sequence:     [0.   0.39 0.13 0.26 0.52]
neg start 1:  []
neg end 1     []
pos start 1:  [ 775  876  890  983 1023 1136]
pos end 1:    [ 818  888  906 1015 1046 1151]
neg start 2:  [1061 1152]
neg end 2     [1089 1234]
pos start 2:  [ 834  925 1115 1278]
pos end 2:    [ 866  962 1134 1404]


In [4]:
# generate toy data: sequence, labels
def gen_toy_data():

    # Generate a sequence with 10 numbers
    sequence_length = 10
    sequence = np.zeros(sequence_length + 1)

    means = [5, 2]                # Define the means for the 4 segments
    segment_lengths = [5, 5]      # Define the lengths of the segments

    # Populate the sequence with segments having different means
    start_index = 1
    for mean, length in zip(means, segment_lengths):
        end_index = start_index + length
        sequence[start_index:end_index] = np.random.normal(loc=mean, scale=0.2, size=length)
        start_index = end_index

    # outlier
    sequence[8] = 8

    # Labels
    neg_start = [2, 7]      # there is no changepoint at point 2, 7, and 8
    neg_end   = [3, 9]      #
    pos_start = [4]         # there can be exactly one changepoint at 4 or 5
    pos_end   = [6]         #

    return sequence, neg_start, neg_end, pos_start, pos_end

# test
toy_sequence, neg_start, neg_end, pos_start, pos_end = gen_toy_data()
print("sequence:  ", toy_sequence)
print("neg start: ", neg_start)
print("neg end    ", neg_end)
print("pos start: ", pos_start)
print("pos end:   ", pos_end)

sequence:   [0.   4.78 5.2  5.06 4.7  4.88 2.33 1.51 8.   2.25 1.83]
neg start:  [2, 7]
neg end     [3, 9]
pos start:  [4]
pos end:    [6]


In [5]:
# Get cumulative sum vectors
def get_cumsum(sequence):
    y = np.cumsum(sequence)
    z = np.cumsum(np.square(sequence))

    y = np.append([0], y)
    z = np.append([0], z)

    return y, z

# test
y, z = get_cumsum(toy_sequence)
print("cumsum vector:", y)
print("cumsum square:", z)

cumsum vector: [ 0.    0.    4.78  9.98 15.04 19.74 24.62 26.95 28.47 36.47 38.72 40.55]
cumsum square: [  0.     0.    22.88  49.91  75.48  97.56 121.41 126.84 129.14 193.14
 198.22 201.55]


In [6]:
# function to create loss value from 'start' to 'end' given cumulative sum vector y (data) and z (square)
def L(start, end, y, z):
    _y = y[end+1] - y[start]
    _z = z[end+1] - z[start]
    return _z - np.square(_y)/(end-start+1)

# test
y, z = get_cumsum(toy_sequence)
print("sequence    : ", toy_sequence)
print("from 1 to 1 : ", L(1, 1,  y, z))
print("from 1 to 2 : ", L(1, 2,  y, z))
print("from 5 to 6 : ", L(5, 6,  y, z))
print("from 1 to 10: ", L(1, 10, y, z))

sequence    :  [0.   4.78 5.2  5.06 4.7  4.88 2.33 1.51 8.   2.25 1.83]
from 1 to 1 :  0.0
from 1 to 2 :  0.08677578448781986
from 5 to 6 :  3.2614392081723373
from 1 to 10:  37.14793999709528


In [7]:
# function to get the list of changepoint from vector tau_star
def trace_back(tau_star):
    tau = tau_star[-1]
    chpnt = np.array([len(tau_star)], dtype=int)
    while tau > 0:
        chpnt = np.append(tau, chpnt)
        tau = tau_star[tau-1]
    return np.append(0, chpnt)

# test
print(trace_back(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])))
print(trace_back(np.array([0, 0, 0, 0, 0, 5, 5, 5, 5, 5])))

[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  5 10]


In [ ]:
# function to get the mean vector of the sequence from the list of changepoint
def get_mean(y, chpnt):
    mean = np.zeros(len(y)-1)
    for i in range(len(chpnt)-1):
        mean[chpnt[i]+1:chpnt[i+1]+1] = (y[chpnt[i+1]+1] - y[chpnt[i]+1])/(chpnt[i+1] - chpnt[i])
    return mean

In [8]:
# function to get T - set of possible changepoint wrt each position
def get_T(sequence_length, neg_start, neg_end, pos_start, pos_end):
    T = []
    T.append([])

    for i in range(1, sequence_length+1):
        tag = 'otherwise'
        for s, e in zip(neg_start, neg_end):
            if(s < i <= e):
                tag = 'inside region'

        for s, e in zip(pos_start, pos_end):
            if(s < i < e):
                tag = 'inside region'
            elif(i == e):
                tag = 'out positive'
                s_pos = s
                e_pos = e

        match tag:
            case "inside region":
                T.append(T[i-1])
            case "out positive":
                T.append(list(range(s_pos, e_pos)))
            case "otherwise":
                T.append(T[i-1]+[i-1])

    return T

# test
toy_sequence, neg_start, neg_end, pos_start, pos_end = gen_toy_data()
print("sequence:  ", toy_sequence)
print("neg start: ", neg_start)
print("neg end    ", neg_end)
print("pos start: ", pos_start)
print("pos end:   ", pos_end)

T = get_T(len(toy_sequence) - 1, neg_start, neg_end, pos_start, pos_end)
print("T:", T)

sequence:   [0.   4.86 4.98 5.3  4.87 4.91 1.91 2.44 8.   2.2  2.08]
neg start:  [2, 7]
neg end     [3, 9]
pos start:  [4]
pos end:    [6]
T: [[], [0], [0, 1], [0, 1], [0, 1, 3], [0, 1, 3], [4, 5], [4, 5, 6], [4, 5, 6], [4, 5, 6], [4, 5, 6, 9]]


In [9]:
# counting errors
def count_items_between(lst, a, b):
    count = sum(1 for item in lst if a <= item < b)
    return count

def error_count(chpnt, neg_start, neg_end, pos_start, pos_end):
    fp_count, fn_count = 0, 0                           # initizlize false positive and false negative

    for s, e in zip(neg_start, neg_end):
        if(count_items_between(chpnt, s, e) > 0):       # number of change is not 0 in negative labels
            fp_count += 1

    for s, e in zip(pos_start, pos_end):
        if(count_items_between(chpnt, s, e) > 1):       # number of change is greater than 1 in positive labels
            fp_count += 1
        elif(count_items_between(chpnt, s, e) == 0):    # number of change is 0 in positive labels
            fn_count += 1

    return fp_count, fn_count

# test
toy_sequence, neg_start, neg_end, pos_start, pos_end = gen_toy_data()
print("neg start: ", neg_start)
print("neg end    ", neg_end)
print("pos start: ", pos_start)
print("pos end:   ", pos_end)

chpnt = [0, 3, 5, 6, 9, 10]
print(chpnt, "   :", error_count(chpnt, neg_start, neg_end, pos_start, pos_end))

chpnt = [0, 1, 2, 6, 8, 9, 10]
print(chpnt, ":",error_count(chpnt, neg_start, neg_end, pos_start, pos_end))

neg start:  [2, 7]
neg end     [3, 9]
pos start:  [4]
pos end:    [6]
[0, 3, 5, 6, 9, 10]    : (0, 0)
[0, 1, 2, 6, 8, 9, 10] : (2, 1)


In [ ]:
# function to plot sequence with labels and changepoints wrt lambda (if provided)
from plotnine import *

def plot_sequence(sequence, neg_start, neg_end, pos_start, pos_end, algorithm='LOPART', chpnt=None, mean=None, lda=None):
    sequence = sequence[1:]

    # Prepare data for plotnine
    data = pd.DataFrame({'point': range(1, len(sequence)+1), 'value': sequence})

    # Create the plot
    plot = (
        ggplot(data, aes(x='point', y='value')) +
        theme_minimal() +
        labs(x='point', y='value') +
        xlim(0, len(sequence) + 1) +
        ylim(np.min(sequence) - 1, np.max(sequence) + 1)
    )

    # Add negative regions
    for start, end in zip(neg_start, neg_end):
        plot += geom_rect(aes(xmin=start, xmax=end, ymin=np.min(sequence) - 1, ymax=np.max(sequence) + 1),
                          fill='pink', alpha=0.2, color='black', size=0)

    # Add positive regions
    for start, end in zip(pos_start, pos_end):
        plot += geom_rect(aes(xmin=start, xmax=end, ymin=np.min(sequence) - 1, ymax=np.max(sequence) + 1),
                          fill='red', alpha=0.2, color='black', size=0)

    # Add data points
    plot += geom_point(color='blue', size=2, alpha=0.7)

    # Add mean line
    if mean is not None:
        mean = mean[1:]
        for i in range(len(chpnt) - 1):
            plot += geom_segment(aes(x=chpnt[i]+0.5, y=mean[chpnt[i]], xend=chpnt[i+1]+0.5, yend=mean[chpnt[i]]),
                            color='green', size=1, alpha=0.7)
    # Add changepoint
    if chpnt is not None:
        for i in range(1, len(chpnt)-1):
            plot += geom_vline(xintercept=chpnt[i]+0.5, linetype='dashed', color='deepskyblue', alpha=0.8)

    # Set the figure title
    if lda is not None:
        plot += ggtitle(algorithm + ' -- lambda = ' + str(lda) + ' -- errors = ' + str(sum(error_count(chpnt, neg_start, neg_end, pos_start, pos_end))))

    # Center the title horizontally
        plot += theme(plot_title=element_text(hjust=0.5))

    # Add legend
    plot += labs(color="Legend")  # Legend title
    plot += guides(color=guide_legend(title="Legend"))  # Legend label

    # return the plot
    return plot

In [ ]:
# function to write a row into a csv file
def write_to_csv(filename, header, row):
    # Check if the file exists
    file_exists = os.path.isfile(filename)

    # Open the CSV file in write mode
    with open(filename, 'a', newline='') as csvfile:
        # Create a CSV writer object
        csv_writer = csv.writer(csvfile)

        # If the file is newly created, write the header row
        if not file_exists:
            csv_writer.writerow(header)

        # Write the new row
        csv_writer.writerow(row)

In [10]:
# OPART given lambda, sequence, y and z
def opart(lda, sequence, y, z):
    sequence_length = len(sequence)-1

    # Set up
    C = np.zeros(sequence_length + 1)
    C[0] = -lda

    # Get tau_star
    tau_star = np.zeros(sequence_length+1, dtype=int)
    for t in range(1, sequence_length+1):

        # get set of possible value
        V = C[:t] + lda + L(1 + np.arange(t), t, y, z)

        # get optimal tau from set V
        last_chpnt = np.argmin(V)

        # update C_i
        C[t] = V[last_chpnt]

        # update tau_star
        tau_star[t] = last_chpnt

    # get set of changepoints
    set_of_chpnt = trace_back(tau_star[1:])

    return set_of_chpnt

# test
toy_sequence, neg_start, neg_end, pos_start, pos_end = gen_toy_data()
sequence_length = len(toy_sequence) - 1
y, z = get_cumsum(toy_sequence)
for lda in [0, 10, 100]:
    print("lambda = %4d: %s" % (lda, opart(lda, toy_sequence, y, z)))

lambda =    0: [ 0  1  2  3  4  5  6  7  8  9 10]
lambda =   10: [ 0  5  7  8 10]
lambda =  100: [ 0 10]


In [11]:
# lopart dynamic algorithm return set of changepoints given lambda, T (set of possible changepoints), sequence, and cumsum vectors
def lopart(lda, T, sequence, y, z):
    sequence_length = len(sequence)-1

    # Set up
    C = np.zeros(sequence_length + 1)
    C[0] = -lda

    # Get tau_star
    tau_star = np.zeros(sequence_length+1, dtype=int)
    for t in range(1, sequence_length+1):

        # get set of possible changepoint
        po_chpnt = T[t]

        # get set of possible value
        V = np.inf * np.ones(sequence_length+1)
        for j in po_chpnt:
            V[j] = C[j] + lda + L(j+1, t, y, z)

        # get optimal tau from set V
        last_chpnt = np.argmin(V)

        # update C_i
        C[t] = V[last_chpnt]

        # update tau_star
        tau_star[t] = last_chpnt

    # get set of changepoints
    set_of_chpnt = trace_back(tau_star[1:])

    return set_of_chpnt